#### 목적 : 데이터베이스가 테이블이 생성되고 json 파일이랑 DB랑 연결되는지 테스트

In [ ]:
# Python 경로 추가
import sys
import os
import json
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from app.database.connection import DatabaseSettings, create_connection
from app.services.data_ingestion_service import create_tables
from app.services.data_insert_DB import insert_books_and_authors

# 데이터베이스 연결 설정
db_settings = DatabaseSettings()
conn = create_connection(db_settings)

if conn:
    # 테이블 생성
    create_tables(conn)
    print('success')

    # JSON 파일 경로
    json_file_path = "../data/scraping/all_book_data.json"

    # JSON 데이터를 테이블에 삽입
    insert_books_and_authors(conn, json_file_path)

    # 데이터 조회 테스트
    def test_connection(conn):
        """테이블 데이터 조회 테스트"""
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM books;")
            rows = cursor.fetchall()
            if rows:
                for row in rows:
                    print(row)
            else:
                print("No data found in 'books' table.")
        except Exception as e:
            print("Error during connection test:", e)
        finally:
            cursor.close()

    test_connection(conn)

    # 연결 종료
    conn.close()
    print("Database connection closed.")
